<a href="https://colab.research.google.com/github/Jahan08/Ambertools-CP2K-MM-QM-Biomolecular-Simulation/blob/main/hERG-Meta-learning-Convolution-Neural-Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 29.9 MB/s eta 0:00:00


In [13]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [15]:
df = pd.read_csv("/content/hERG_IC50_nM_new.csv")
df

,assay_chembl_id,assay_description,assay_type,canonical_smiles,document_journal,molecule_chembl_id,Value,Activity
0,CHEMBL841079,Inhibition of hERG currents Kv11.1,T,O=C1NCCN1CCN1CCC(c2cn(-c3ccc(F)cc3)c3ccc(Cl)cc...,J Med Chem,CHEMBL12713,14.0,No
1,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,O=C(CCCN1CC=C(n2c(=O)[nH]c3ccccc32)CC1)c1ccc(F...,J Med Chem,CHEMBL1108,32.2,No
2,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,COc1ccc(CCN(C)CCCC(C#N)(c2ccc(OC)c(OC)c2)C(C)C...,J Med Chem,CHEMBL6966,143.0,No
3,CHEMBL877203,K+ channel blocking activity in Chinese hamste...,F,CCCCN(CCCC)CCC(O)c1cc2c(Cl)cc(Cl)cc2c2cc(C(F)(...,J Med Chem,CHEMBL1107,196.0,No
4,CHEMBL691014,K+ channel blocking activity in human embryoni...,F,CCOC(=O)N1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,J Med Chem,CHEMBL998,173.0,No
...,...,...,...,...,...,...,...,...
2963,CHEMBL5048865,Inhibition of hERG by patch clamp method,T,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(C)(=O)=O)c...,J Med Chem,CHEMBL5081517,44000.0,Yes
2964,CHEMBL5048865,Inhibition of hERG by patch clamp method,T,CCOP(=O)(Cn1ccc(NC(=O)c2cc(Oc3ccc(S(=O)(=O)N4C...,J Med Chem,CHEMBL5072442,33000.0,Yes
2965,CHEMBL5049389,Inhibition of hERG,T,Cc1nc(C)c([C@H](OC(C)(C)C)C(=O)O)c(N2CCC(C)(C)...,J Med Chem,CHEMBL5093378,670.0,No
2966,CHEMBL5050750,Inhibition of human ERG,T,Cc1cnc(Nc2ccnn2C)nc1-c1cc2n(c1)C(=O)N([C@H](CO...,ACS Med Chem Lett,CHEMBL5070887,14.0,No
